In [1]:
%cd /home/maximilianheil/fungiclef-2024/

/home/maximilianheil/fungiclef-2024


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pyspark.sql.functions as f
import pyspark.ml as ml
from pyspark.ml.feature import Tokenizer
import fungiclef.embedding.transforms as trans
from fungiclef.utils import get_spark

In [4]:
spark = get_spark(**{
    "spark.sql.parquet.enableVectorizedReader": False, 
})
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 22:21:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/19 22:21:47 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/dev/dev_train/"

#gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/"
#input_folder = f"DF20"



dev_df = spark.read.parquet(gcs_parquet_path)
dev_df.printSchema()
dev_df.count()

root
 |-- image_path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- observationID: long (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- taxonID: double (nullable = true)
 |-- scientificName: string (nullable = true)
 |-- kingdom: string (nullable = true)
 |-- phylum: string (nullable = true)
 |-- class: string (nullable = true)
 |-- order: string (nullable = true)
 |-- family: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- specificEpithet: string (nullable = true)
 |-- taxonRank: string (nullable = true)
 |-- species: string (nullable = true)
 |-- level0Gid: string (nullable = true)
 |-- level0Name: string (nullable = true)
 |-- level1Gid: string (nullable = true)
 |-- level1Name: string (nullable = true)
 |-- level2Gid: string (nullable = true)
 |-- level2Name: string (nullable = tr

3723

In [6]:
ignore_cols = ['class', 'phylum', 'Latitude', 'Longitude', 'CoorUncert', 'txt_data', 'taxonRank', 'class_id', 'kingdom', 'image_path', 'data', 'observationID', 'year', 'ImageUniqueID', 'rightsHolder', 'day', 'countryCode', 'taxonID', 'level0Gid', 'level0Name', 'level1Gid', 'level2Gid']
relevant_columns = [col for col in  dev_df.columns if col not in ignore_cols]
relevant_columns

['month',
 'locality',
 'scientificName',
 'order',
 'family',
 'genus',
 'specificEpithet',
 'species',
 'level1Name',
 'level2Name',
 'Substrate',
 'Habitat',
 'MetaSubstrate',
 'poisonous']

In [7]:
def cast_floats_to_int(df, list_of_cols):
    for col in list_of_cols:
        df = df.withColumn(col, dev_df[col].cast('int'))
    return df

dev_df = cast_floats_to_int(dev_df, ['month'])

In [8]:
# concat all txt data in one col txt_data

transformed_cols = [f.concat(f.lit(f"{col_name} "), f.col(col_name).cast("string")).alias(col_name) for col_name in relevant_columns]

dev_df = dev_df.withColumn("txt_data", f.concat_ws(", ", *transformed_cols))

dev_df.select("txt_data").show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 txt_data | month 9, locality Smørmosen, scientificName Russula nigricans (Bull.) Fr., order Russulales, family Russulaceae, genus Russula, specificEpithet nigricans, species Russula adusta, level1Name Hovedstaden, level2Name Herlev, Substrate soil, Habitat bog, MetaSubstrate jord, poisonous 0 
only showing top 1 row



In [9]:
tokenizer = Tokenizer(outputCol="txt_token")
tokenizer.setInputCol("txt_data")

tokenized_df = tokenizer.transform(dev_df).select('txt_token', 'txt_data')

tokenized_df = tokenized_df.withColumn("len_txt_token", f.size(f.col("txt_token")))
tokenized_df = tokenized_df.withColumn("raw_len", f.length(f.col("txt_data")))

tokenized_df.sort('len_txt_token', ascending=False).show(10, vertical=False)

+--------------------+--------------------+-------------+-------+
|           txt_token|            txt_data|len_txt_token|raw_len|
+--------------------+--------------------+-------------+-------+
|[month, 9,, local...|month 9, locality...|           47|    408|
|[month, 9,, local...|month 9, locality...|           47|    381|
|[month, 9,, local...|month 9, locality...|           47|    408|
|[month, 9,, local...|month 9, locality...|           47|    381|
|[month, 7,, local...|month 7, locality...|           47|    408|
|[month, 9,, local...|month 9, locality...|           47|    412|
|[month, 7,, local...|month 7, locality...|           47|    408|
|[month, 11,, loca...|month 11, localit...|           46|    379|
|[month, 11,, loca...|month 11, localit...|           46|    379|
|[month, 9,, local...|month 9, locality...|           46|    380|
+--------------------+--------------------+-------------+-------+
only showing top 10 rows



In [ ]:
# concat all txt data in one col txt_data

transformed_cols = [f.concat(f.lit(f"{col_name} "), f.col(col_name).cast("string")).alias(col_name) for col_name in relevant_columns]

dev_df = dev_df.withColumn("txt_data", f.concat_ws(", ", *relevant_columns))

dev_df.select("txt_data").show(1, vertical=True, truncate=False)

In [10]:
# Init DINOv2 wrapper
clip = trans.WrappedCLIP(input_cols=["data","txt_data"], output_col="transformed_data")

# Create Pipeline
pipeline = ml.Pipeline(stages=[clip]) #, dctn

# Fit pipeline to DF
model = pipeline.fit(dev_df)

# Apply the model to transform the DF
transformed_df = model.transform(dev_df).cache()

24/04/19 22:22:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)

Token indices sequence length is longer than the specified maximum sequence length for this model (85 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (125 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (93 > 77). Running this sequence through the model will result in indexing errors
{'input_ids': tensor([[49406,  1924,   279,   267,  9202,   696, 39144,    82, 18449,   909,
          8479,   267,  4338,  1788,  1981, 14562,   647,  1898,   718,   664,
           553,   570,  1157,   533,   263,   532,   522,   330,  5376,  7343,
         19837,   267,  8796

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/ml/functions.py", line 799, in predict
    preds = predict_fn(*multi_inputs)
  File "/home/maximilianheil/fungiclef-2024/fungiclef/embedding/transforms.py", line 192, in predict
    text_features = outputs.text_embeds
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/clip/modeling_clip.py", line 1117, in forward
    text_outputs = self.text_model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args

In [88]:
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)

+-----------------+--------------------+--------------------+
|    ImageUniqueID|             species|    transformed_data|
+-----------------+--------------------+--------------------+
| 2237920504-77555|      Russula adusta|[0.009258113, 0.0...|
|  2237966683-5021|      Russula adusta|[0.019642368, 0.0...|
| 2237968009-79449|      Russula adusta|[0.0032701602, 0....|
| 2238149799-82085|Neoboletus luridi...|[-0.0048931465, 0...|
|  2238150033-7827|       Imleria badia|[-0.0015484869, 0...|
|2238171177-306291|  Lactarius blennius|[0.003171264, 0.0...|
| 2238209494-84510|Neoboletus luridi...|[0.022397494, 0.0...|
| 2238331048-11804|  Russula ochroleuca|[-0.014820853, 0....|
| 2238428423-15997|  Russula ochroleuca|[0.024092644, 0.0...|
| 2238455499-91533|  Lactarius blennius|[0.0208195, 0.047...|
+-----------------+--------------------+--------------------+
only showing top 10 rows



In [ ]:

# Init Descrite Cosine Transform wrapper
dctn = trans.DCTN(input_col="transformed_data", output_col="dctn_data")

# Create Pipeline
pipeline = ml.Pipeline(stages=[dino, dctn]) #, dctn

# Fit pipeline to DF
model = pipeline.fit(dev_subset_df)

# Apply the model to transform the DF
transformed_df = model.transform(dev_subset_df).cache()

# Show results
transformed_df.select(["ImageUniqueID", "species", "transformed_data"]).show(n=10)